<a href="https://colab.research.google.com/github/ZohebAbai/Tiny-ImageNet-Challenge/blob/master/TinyImageNet_Network_2_with_additional_methods_without_clr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Submitted by : Zoheb Abai (EIP 3 Batch 1)**

# Installing libraries and Downloading Dataset

In [0]:
# Installing and importing keras
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
## Downloading the dataset 
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2019-04-05 07:16:12--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  61.1MB/s    in 5.0s    

2019-04-05 07:16:17 (47.7 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [0]:
# Unzipping it 
!unzip -qq 'tiny-imagenet-200.zip'
!ls

sample_data  tiny-imagenet-200	tiny-imagenet-200.zip


In [0]:
# Mounting google drive for saving models
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!pip install git+https://github.com/keras-team/keras-contrib.git

In [0]:
# Importing important libraries
import numpy as np
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
#from keras_contrib.callbacks import CyclicLR
import imgaug as ia
from imgaug import augmenters as iaa

# random seed
np.random.seed(seed=101)
ia.seed(101)

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# input image dimensions
img_height = 64
img_width = 64
channels = 3

# Hyperparameters
batch_size = 128
num_classes = 200
epochs = 36
num_train = 100000
num_validation = 10000

# Callbacks
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.000001)
#clr = CyclicLR(base_lr=0.00001, max_lr=0.001, step_size=852., mode='triangular2') #Cyclic learning rate
checkpointer = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/top_acc_weights.hdf5", verbose=1, save_best_only=True, monitor="val_acc")

In [0]:
# Dropping the annotations from txt file as its not required for this project
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727


# Image Augmentation

In [0]:
# Defining Customized Imagedatagenerator using imgaug library
def CustomImageDataGen(input_img):
  # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
  # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
  # image.
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  
  seq = iaa.Sequential([
      iaa.Fliplr(0.5), # horizontal flips
      iaa.Flipud(0.2), # vertical flips
      
      # Small gaussian blur with random sigma between 0 and 0.5.
      # But we only blur about 50% of all images.
      sometimes(iaa.GaussianBlur(sigma=(0, 2.0))),
      
      # crop images by -10% to 20% of their height/width
      sometimes(iaa.CropAndPad(
          percent=(-0.1, 0.2),
          pad_mode=ia.ALL,
          pad_cval=(0, 255)
        )),
      
      # Apply affine transformations to some of the images
      # - scale to 80-120% of image height/width (each axis independently)
      # - translate by -20 to +20 relative to height/width (per axis)
      # - rotate by -45 to +45 degrees
      # - shear by -16 to +16 degrees
      # - order: use nearest neighbour or bilinear interpolation (fast)
      # - mode: use any available mode to fill newly created pixels
      #         see API or scikit-image for which modes are available
      # - cval: if the mode is constant, then use a random brightness
      #         for the newly created pixels (e.g. sometimes black,
      #         sometimes white)
      sometimes(iaa.Affine(
          scale={"x": (0.8, 1.5), "y": (0.8, 1.5)},
          translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
          rotate=(-45, 45),
          shear=(-16, 16),
          order=[0, 1],
          cval=(0, 255),
          mode=ia.ALL
      )),
      
      #drop 2-5% percent of the original size, leading to large dropped
      # rectangles.
      sometimes(iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    )),
      # Add gaussian noise.
      # For 50% of all images, we sample the noise once per pixel.
      # For the other 50% of all images, we sample the noise per pixel AND
      # channel. This can change the color (not only brightness) of the
      # pixels.
      #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
      
      # Make some images brighter and some darker.
      # In 20% of all cases, we sample the multiplier once per channel,
      # which can end up changing the color of the images.
      sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.2)),
      
      # Improve or worsen the contrast of images.
      sometimes(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)),  
      
      # Converts images from colorspace to grayscale and mixes with the original 
      # image using alpha A
      #sometimes(iaa.Grayscale(alpha=(0.0, 1.0))),
     ],
     # do all of the above augmentations in random order
     random_order = True) # apply augmenters in random order
  
  output_img = seq.augment_image(input_img)
  return output_img

train_datagen = ImageDataGenerator(preprocessing_function = CustomImageDataGen)
valid_datagen = ImageDataGenerator()

In [0]:
# Training set generator
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, 
                                                    #shuffle=False, #Use only for viewing predictions after 100 epochs
                                                    seed=101)

Found 100000 images belonging to 200 classes.


In [0]:
# Validation set generator
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

Found 10000 images belonging to 200 classes.


# Model Building and Compilation

### Below is the custom Resnet Model, where architecture is inspired from Wide-Resnet and Resnet-18 keras models. As required for the project followings have not been used :
1. 1x1 for an increasing number of channels
2. dropout
3. fully connected layers
4. test dataset for training
5. pre-trained model/weights
6. someone else's code 

### Note : For running the new model after 12 hours of run, don't run beyond this, as the saved model contains model architecture, weights and optimizer.

In [0]:
# Model building
input = Input(shape=(img_height, img_width, channels))

# Block 1
layer0 = Conv2D(32, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(input)
layer0 = BatchNormalization()(layer0)
layer0 = Activation('relu')(layer0)

skip_connection_1 = layer0

# Block 2

layer1 = Conv2D(128, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer0)
layer1 = BatchNormalization()(layer1)
layer1 = Activation('relu')(layer1)

layer2 = Conv2D(128, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Activation('relu')(layer2)

layer3 = Conv2D(128, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)

layer4 = Conv2D(128, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Activation('relu')(layer4)

layer5 = concatenate([skip_connection_1, layer4])
layer5 = BatchNormalization()(layer5)
layer5 = Activation('relu')(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_2 = layer5

# Block 3

layer6 = Conv2D(256, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer5)
layer6 = BatchNormalization()(layer6)
layer6 = Activation('relu')(layer6)

layer7 = Conv2D(256, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer6)
layer7 = BatchNormalization()(layer7)
layer7 = Activation('relu')(layer7)

layer8 = Conv2D(256, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer7)
layer8 = BatchNormalization()(layer8)
layer8 = Activation('relu')(layer8)

layer9 = Conv2D(256, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer8)
layer9 = BatchNormalization()(layer9)
layer9 = Activation('relu')(layer9)

layer10 = concatenate([skip_connection_2, layer9])
layer10 = BatchNormalization()(layer10)
layer10 = Activation('relu')(layer10)
layer10 = MaxPooling2D(pool_size=(2, 2))(layer10)

skip_connection_3 = layer10


# Block 4

layer11 = Conv2D(512, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer10)
layer11 = BatchNormalization()(layer11)
layer11 = Activation('relu')(layer11)

layer12 = Conv2D(512, (3,3), padding='same', kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer11)
layer12 = BatchNormalization()(layer12)
layer12 = Activation('relu')(layer12)

layer13 = Conv2D(512, (3,3), padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer12)
layer13 = BatchNormalization()(layer13)
layer13 = Activation('relu')(layer13)

layer14 = Conv2D(512, (3,3), padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer13)
layer14 = BatchNormalization()(layer14)
layer14 = Activation('relu')(layer14)

layer15 = concatenate([skip_connection_3, layer14)
layer15 = BatchNormalization()(layer15)
layer15 = Activation('relu')(layer15)
layer15 = MaxPooling2D(pool_size=(2, 2))(layer15)


#Layer 16
layer16 = Conv2D(num_classes, (1,1), padding='same',kernel_initializer="he_uniform",kernel_regularizer=l2(1e-4))(layer15)
layer16 = GlobalAveragePooling2D()(layer16)

#Output Layer
output = Activation('softmax')(layer16)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Model Summary
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              #optimizer= RMSprop(epsilon=1e-08),
              optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [0]:
# Fit the Model
model.fit_generator(train_generator,
                    epochs=epochs,
                    steps_per_epoch= num_train // batch_size,
                    validation_steps= num_validation // batch_size,
                    validation_data=validation_generator,
                    verbose=1, callbacks=[lr_reducer, checkpointer]
                   )

Instructions for updating:
Use tf.cast instead.
Epoch 1/36
781/781 [==============================] - 1198s 2s/step - loss: 5.6243 - acc: 0.0484 - val_loss: 5.0938 - val_acc: 0.0997

Epoch 00001: val_acc improved from -inf to 0.09966, saving model to /content/drive/My Drive/Colab Notebooks/top_acc_weights.hdf5
Epoch 2/36
781/781 [==============================] - 1183s 2s/step - loss: 5.0986 - acc: 0.1003 - val_loss: 4.7026 - val_acc: 0.1404

Epoch 00002: val_acc improved from 0.09966 to 0.14040, saving model to /content/drive/My Drive/Colab Notebooks/top_acc_weights.hdf5
Epoch 3/36
781/781 [==============================] - 1181s 2s/step - loss: 4.7747 - acc: 0.1399 - val_loss: 4.3459 - val_acc: 0.1930

Epoch 00003: val_acc improved from 0.14040 to 0.19297, saving model to /content/drive/My Drive/Colab Notebooks/top_acc_weights.hdf5
Epoch 4/36
781/781 [==============================] - 1181s 2s/step - loss: 4.5230 - acc: 0.1738 - val_loss: 4.2982 - val_acc: 0.1996

Epoch 00004: val_ac

### After 12 hours of running and saving the model, I shall run the new model from here after skipping the above part. 

In [0]:
# load the model after 12 hours
from keras.models import load_model
new_model = load_model("/content/drive/My Drive/Colab Notebooks/top_acc_weights.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# fit the model
checkpointer_new = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/new_top_acc_weights.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
new_model.fit_generator(train_generator,
                        epochs=epochs,
                        steps_per_epoch= num_train // batch_size,
                        validation_steps= num_validation // batch_size,
                        validation_data=validation_generator,
                        verbose=1, callbacks=[lr_reducer, checkpointer_new]
                       )

Epoch 1/36
781/781 [==============================] - 1253s 2s/step - loss: 2.4292 - acc: 0.5250 - val_loss: 2.4123 - val_acc: 0.5427

Epoch 00001: val_acc improved from -inf to 0.54267, saving model to /content/drive/My Drive/Colab Notebooks/new_top_acc_weights.hdf5
Epoch 2/36
781/781 [==============================] - 1240s 2s/step - loss: 2.4115 - acc: 0.5306 - val_loss: 2.4135 - val_acc: 0.5383

Epoch 00002: val_acc did not improve from 0.54267
Epoch 3/36
781/781 [==============================] - 1237s 2s/step - loss: 2.3847 - acc: 0.5341 - val_loss: 2.4046 - val_acc: 0.5343

Epoch 00003: val_acc did not improve from 0.54267
Epoch 4/36
781/781 [==============================] - 1238s 2s/step - loss: 2.3610 - acc: 0.5407 - val_loss: 2.4551 - val_acc: 0.5365

Epoch 00004: val_acc did not improve from 0.54267
Epoch 5/36
781/781 [==============================] - 1238s 2s/step - loss: 2.3444 - acc: 0.5422 - val_loss: 2.3679 - val_acc: 0.5469

Epoch 00005: val_acc improved from 0.54267

### After 24 hours of running and saving the model, I shall run the model from here after skipping the above parts. 

In [0]:
# load the model after 24 hours
from keras.models import load_model
new_model = load_model("/content/drive/My Drive/Colab Notebooks/new_top_acc_weights.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# fit the model
checkpointer_new = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/final_top_acc_weights.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
new_model.fit_generator(train_generator,
                        epochs=30,
                        steps_per_epoch= num_train // batch_size,
                        validation_steps= num_validation // batch_size,
                        validation_data=validation_generator,
                        verbose=1, callbacks=[lr_reducer, checkpointer_new]
                       )

Epoch 1/30
781/781 [==============================] - 1213s 2s/step - loss: 1.7795 - acc: 0.6641 - val_loss: 2.2226 - val_acc: 0.5822

Epoch 00001: val_acc improved from -inf to 0.58223, saving model to /content/drive/My Drive/Colab Notebooks/final_top_acc_weights.hdf5
Epoch 2/30
781/781 [==============================] - 1199s 2s/step - loss: 1.7690 - acc: 0.6669 - val_loss: 2.2461 - val_acc: 0.5872

Epoch 00002: val_acc improved from 0.58223 to 0.58722, saving model to /content/drive/My Drive/Colab Notebooks/final_top_acc_weights.hdf5
Epoch 3/30
781/781 [==============================] - 1196s 2s/step - loss: 1.7522 - acc: 0.6704 - val_loss: 2.2255 - val_acc: 0.5860

Epoch 00003: val_acc did not improve from 0.58722
Epoch 4/30
781/781 [==============================] - 1196s 2s/step - loss: 1.7394 - acc: 0.6740 - val_loss: 2.2715 - val_acc: 0.5836

Epoch 00004: val_acc did not improve from 0.58722
Epoch 5/30
781/781 [==============================] - 1195s 2s/step - loss: 1.7318 - ac

### After 34 hours with a total of 100 epochs we get a validation accuracy of 60.76% which is currently saturated as it reached here from accuracy of 59.53% of last run after 30 epochs.

# Model Run on Oversampling the Misclassified Training Data

### Before further running the model, we shall now divide the training set into two different sets of good and bad images, which represents the images correctly classified and incorrectly classified respectively. And further train our model on it for improving val accuracy.

In [0]:
# load the model after 34 hours
from keras.models import load_model
final_model = load_model("/content/drive/My Drive/Colab Notebooks/final_top_acc_weights.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
#Create directories and foldrs for good and bad images
from os import makedirs
from os.path import join

for _class in validation_generator.class_indices.keys():
  makedirs(join('good_images',_class))
  makedirs(join('bad_images', _class))

In [0]:
#Prediction on training dataset
pred_train=final_model.predict_generator(train_generator, steps= np.ceil(num_train/batch_size), verbose=1)
pred_train_class_indices=np.argmax(pred_train,axis=1)

782/782 [==============================] - 627s 802ms/step


In [0]:
# Label the training dataset
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in pred_train_class_indices]

In [0]:
# Copy the respective images to good and bad images folders according to their classification
import shutil
k, num_good, num_bad = 0, 0, 0
for i in range(200):
  for j in range(500):
    if pred_train_class_indices[k+j] == i and k+j != 100000:
      num_good += 1
      shutil.copy2('tiny-imagenet-200/train/'+labels[i]+'/images/'+labels[i]+'_'+str(j)+'.JPEG', 'good_images/'+labels[i])
    else:
      num_bad += 1
      shutil.copy2('tiny-imagenet-200/train/'+labels[i]+'/images/'+labels[i]+'_'+str(j)+'.JPEG', 'bad_images/'+labels[i])
      
  k += 500

In [0]:
# Creating good and bad training image dataset
good_train = train_datagen.flow_from_directory( r'./good_images/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, 
                                                    seed=101)
bad_train = train_datagen.flow_from_directory( r'./bad_images/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, 
                                                    seed=101)

Found 72831 images belonging to 200 classes.
Found 27169 images belonging to 200 classes.


In [0]:
# fit the model
checkpointer_extended = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/extended_model_weights.hdf5", verbose=1, save_best_only=True, monitor="val_acc")

for i in range(10):
  print("\n Epoch for 1 good : 9 bad training ==> ", i+1)
  final_model.fit_generator(good_train,
                            epochs=1,
                            steps_per_epoch= np.ceil(num_good/batch_size),
                            validation_steps= np.ceil(num_validation/batch_size),
                            validation_data=validation_generator,
                            verbose=1, callbacks=[lr_reducer, checkpointer_extended]
                            )
  
  final_model.fit_generator(bad_train,
                            epochs=9,
                            steps_per_epoch= np.ceil(num_bad/batch_size),
                            validation_steps= np.ceil(num_validation/batch_size),
                            validation_data=validation_generator,
                            verbose=1, callbacks=[lr_reducer, checkpointer_extended]
                            )


 Epoch for 1 good : 9 bad training ==>  1
Epoch 1/1
569/569 [==============================] - 911s 2s/step - loss: 1.4871 - acc: 0.7303 - val_loss: 2.1628 - val_acc: 0.6062

Epoch 00001: val_acc improved from -inf to 0.60620, saving model to /content/drive/My Drive/Colab Notebooks/extended_model_weights.hdf5
Epoch 1/9
213/213 [==============================] - 360s 2s/step - loss: 1.6545 - acc: 0.6866 - val_loss: 2.1820 - val_acc: 0.6049

Epoch 00001: val_acc did not improve from 0.60620
Epoch 2/9
213/213 [==============================] - 358s 2s/step - loss: 1.6144 - acc: 0.6980 - val_loss: 2.1887 - val_acc: 0.5985

Epoch 00002: val_acc did not improve from 0.60620
Epoch 3/9
213/213 [==============================] - 357s 2s/step - loss: 1.5911 - acc: 0.7012 - val_loss: 2.1844 - val_acc: 0.5987

Epoch 00003: val_acc did not improve from 0.60620
Epoch 4/9
213/213 [==============================] - 360s 2s/step - loss: 1.5603 - acc: 0.7117 - val_loss: 2.1847 - val_acc: 0.6008

Epoch 

### The process didn't help that much to improve val accuracy as model accuracy had completely saturated.

# Model Run on Weighting the Classes with Low Precision

We use it so that, if you miss-classify classA the loss will be n times more than miss-classifying classB and so on..

In [0]:
# load the model after 46 hours
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/Colab Notebooks/extended_model_weights.hdf5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# Evaluation
score= extended_model.evaluate_generator(validation_generator, steps= np.ceil(num_validation/batch_size), verbose=1)
print("Validation Loss : ", score[0])
print("Validation Accuracy:", score[1]*100, "%")

79/79 [==============================] - 43s 540ms/step
Validation Loss :  2.140896961593628
Validation Accuracy: 61.129999999999995 %


In [0]:
#Prediction
pred=extended_model.predict_generator(validation_generator, steps= np.ceil(num_validation/batch_size), verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

79/79 [==============================] - 43s 545ms/step


In [0]:
# Predicted class indices of 1st 10 val images
predicted_class_indices[:10]

array([107,  15,  83,  81, 168, 161, 147, 172, 145,  10])

In [0]:
# True class indices of 1st 10 val images
validation_generator.classes[:10]

[107, 139, 140, 69, 69, 161, 147, 73, 145, 39]

In [0]:
# Predicted classes from their indices
labels = (validation_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions[:10]

['n03444034',
 'n01944390',
 'n02948072',
 'n02917067',
 'n04507155',
 'n04399382',
 'n04179913',
 'n04560804',
 'n04146614',
 'n01784675']

In [0]:
# Validation class names from words.txt
import os
class_to_name = dict()
file = open('tiny-imagenet-200/words.txt','r')
data= file.readlines()
for line in data:
  words = line.strip('\n').split('\t')
  class_to_name[words[0]] = words[1].split(',')[0]
file.close()

In [0]:
# Asserting Validation Class names from words.txt
validation_class_names={}
for _class in validation_generator.class_indices.keys():
  validation_class_names.update({_class : class_to_name[_class]})

In [0]:
# Classification Report of val classes
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(validation_generator.classes, predicted_class_indices,
                            #target_names=validation_generator.class_indices.keys(),
                            target_names=validation_class_names.values(),
                            digits=4))

                          precision    recall  f1-score   support

                goldfish     0.7736    0.8200    0.7961        50
European fire salamander     0.8913    0.8200    0.8542        50
                bullfrog     0.6250    0.6000    0.6122        50
             tailed frog     0.5652    0.5200    0.5417        50
      American alligator     0.6042    0.5800    0.5918        50
         boa constrictor     0.4681    0.4400    0.4536        50
               trilobite     0.6923    0.9000    0.7826        50
                scorpion     0.6429    0.5400    0.5870        50
             black widow     0.7241    0.8400    0.7778        50
               tarantula     0.6731    0.7000    0.6863        50
               centipede     0.5962    0.6200    0.6078        50
                   goose     0.7805    0.6400    0.7033        50
                   koala     0.7885    0.8200    0.8039        50
               jellyfish     0.6786    0.7600    0.7170        50
         

### We consider the classes with precision lower than 45% to be weighted reciprocal times to its ratio with 45%. We consider weights of classes with precision above 45% as 1. The reason behind this is our average precison of 60% and most of the classes lies within +-15% of it.




### 28 classes that have precision less than 45%

27 - Labrador Retriever,
29 - Standard Poodle,
48 - Hog,
62 - Apron,
64 - Bannister,
77 - Bow-tie,
80 - Bucket,
94 - Convertible,
99 - Drumstick,
100 - Dumbbell,
106 - GasMask,
120 - Miniskirt,
131 - Plunger,
132 - Pole,
135 - Pop Bottle,
136 - Potter's Wheel,
138 - Punching bag,
151 - Space Heater,
159 - Syringe,
160 - Teapot,
167- Turnstile,
168 - Umbrella,
172 - WaterJug,
175 - Wooden Spoon,
179 - Ice cream,
180 -Ice Lolly,
197 - Lakeside,
198- seashore

In [0]:
# Confusion Matrix of Wooden Spoon class
confusion_matrix(validation_generator.classes, predicted_class_indices)[175]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  1,
        1,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  2, 17,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  0,
        0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1])

In [0]:
# Confusion Matrix of Monarch Class
confusion_matrix(validation_generator.classes, predicted_class_indices)[44]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  2,  1,  0,  0,  0,  1, 44,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [0]:
# Class Weight dictionary
class_weights = {0:1,1:1,2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1,10:1,
                 11:1,12:1,13:1,14:1,15:1,16:1,17:1,18:1,19:1,20:1,
                 21:1,22:1,23:1,24:1,25:1,26:1,27:1.12,28:1,29:1.01,30:1,
                 31:1,32:1,33:1,34:1,35:1,36:1,37:1,38:1,39:1,40:1,
                 41:1,42:1,43:1,44:1,45:1,46:1,47:1,48:1.06,49:1,50:1,
                 51:1,52:1,53:1,54:1,55:1,56:1,57:1,58:1,59:1,60:1,
                 61:1,62:1.15,63:1,64:1.19,65:1,66:1,67:1,68:1,69:1,70:1,
                 71:1,72:1,73:1,74:1,75:1,76:1,77:1.10,78:1,79:1,80:1.27,
                 81:1,82:1,83:1,84:1,85:1,86:1,87:1,88:1,89:1,90:1,
                 91:1,92:1,93:1,94:1,95:1,96:1,97:1,98:1,99:1.37,100:1.18,
                 101:1,102:1,103:1,104:1,105:1,106:1.02,107:1,108:1,109:1,110:1,
                 111:1,112:1,113:1,114:1,115:1,116:1,117:1,118:1,119:1,120:1.06,
                 121:1,122:1,123:1,124:1,125:1,126:1,127:1,128:1,129:1,130:1,
                 131:1.49,132:1.52,133:1,134:1,135:1.31,136:1.06,137:1,138:1.33,139:1,140:1,
                 141:1,142:1,143:1,144:1,145:1,146:1,147:1,148:1,149:1,150:1,
                 151:1.21,152:1,153:1,154:1,155:1,156:1,157:1,158:1,159:1.43,160:1.21,
                 161:1,162:1,163:1,164:1,165:1,166:1,167:1.05,168:1.31,169:1,170:1,
                 171:1,172:1.26,173:1,174:1,175:1.51,176:1,177:1,178:1,179:1.16,180:1.02,
                 181:1,182:1,183:1,184:1,185:1,186:1,187:1,188:1,189:1,190:1,
                 191:1,192:1,193:1,194:1,195:1,196:1,197:1.26,198:1.07,199:1,
                }

In [0]:
# fit the model
checkpointer_weighted = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/weighted_model.hdf5", verbose=1, save_best_only=True, monitor="val_acc")
extended_model.fit_generator(train_generator,
                             epochs=10,
                             steps_per_epoch= np.ceil(num_train/batch_size),
                             validation_steps= np.ceil(num_validation/batch_size),
                             validation_data=validation_generator,
                             verbose=1, callbacks=[lr_reducer, checkpointer_weighted],
                             class_weight= class_weights,
                             )

Epoch 1/10
782/782 [==============================] - 1210s 2s/step - loss: 1.4202 - acc: 0.7610 - val_loss: 2.1348 - val_acc: 0.6107

Epoch 00001: val_acc improved from -inf to 0.61070, saving model to /content/drive/My Drive/Colab Notebooks/weighted_model.hdf5
Epoch 2/10
782/782 [==============================] - 1191s 2s/step - loss: 1.4117 - acc: 0.7616 - val_loss: 2.1412 - val_acc: 0.6084

Epoch 00002: val_acc did not improve from 0.61070
Epoch 3/10
782/782 [==============================] - 1192s 2s/step - loss: 1.3996 - acc: 0.7656 - val_loss: 2.1261 - val_acc: 0.6121

Epoch 00003: val_acc improved from 0.61070 to 0.61210, saving model to /content/drive/My Drive/Colab Notebooks/weighted_model.hdf5
Epoch 4/10
782/782 [==============================] - 1191s 2s/step - loss: 1.3864 - acc: 0.7701 - val_loss: 2.1379 - val_acc: 0.6098

Epoch 00004: val_acc did not improve from 0.61210
Epoch 5/10
782/782 [==============================] - 1192s 2s/step - loss: 1.3818 - acc: 0.7703 - va

### I tried every kind of hyperparameters I could and this is the best I could manage. I ran for more than 50 epochs for class_weights approach but after few initial jumps in val acc it stagnates. So finalized with 10 epochs.

# Top Validation Accuracy : 61.58%